# 4.文本与字节序列
## 4.1 字符问题
1. Unicode
    + 字符的标识-码位字符的标识，即码位，是0~1114111 的数字（十进制），在Unicode 标准中以4~6个十六进制数字表示，而且加前缀“U+”。例如，字母A的码位是U+0041，欧元符号的码位是U+20AC
    + 字符的具体表述取决于所用的编码，即utf-8、utf-16。在 UTF-8 编码中，A (U+0041）的码位编码成单个字节\x41，而在 UIF-16LE 编码中编码成两个字节\x41\x00。
2. 编码:encode，解码:decode

## 4.2 字节概要
1. bytes 与 bytearray对象的各个元素都是介于0~255之间的整数
2. 二进制序列的切片始终都是同一类型的二进制序列，包括长度为1的切片
3. [bytes VS bytearray](https://stackoverflow.com/questions/62903377/python3-bytes-vs-bytearray-and-converting-to-and-from-strings?newreg=4d248673031847c88f49f59faf8cb372)
    bytes 不可变、bytearray可变
4. 构建字节序列实例实例，[bytes([source[, encoding[, errors]]])](https://www.runoob.com/python3/python3-func-bytes.html)、bytearray([source[, encoding[, errors]]]),将对象转换为字节对象，或创建指定大小的空字节对象。
    + 如果 source 为整数，则返回一个长度为 source 的初始化数组；
    + 如果 source 为字符串，则按照指定的 encoding 将字符串转换为字节序列；
    + 如果 source 为可迭代类型，则元素必须为[0 ,255] 中的整数；
    + 如果 source 为与 buffer 接口一致的对象，则此对象也可以被用于初始化 bytearray。
    + 如果没有输入任何参数，默认就是初始化数组为0个元素。


In [20]:
cafe  = bytes('café', encoding='utf-8')
print(cafe)
print(cafe[0])#bytes对象的元素为整数
print(cafe[:1])#bytes的切片始终为bytes对象
print(cafe[0]==cafe[:1])
cafe_arr = bytearray(cafe)
print(cafe_arr)
print(cafe_arr[:1])#bytearray 对象的切片仍为bytearray
try:
    cafe[0] = 97
except Exception as e:
    print(e)#bytes类型不可变

cafe_arr[0] = 97
print(cafe_arr)

b'caf\xc3\xa9'
99
b'c'
False
bytearray(b'caf\xc3\xa9')
bytearray(b'c')
'bytes' object does not support item assignment
bytearray(b'aaf\xc3\xa9')


In [21]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])#指定类型h
octets = bytes(numbers)
print(octets)

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'


5. 结构体和内存视图
    + struct 模块提供了一些函数，把打包的字节序列转换成不同类型字段组成的元组，还有一些函数用于执行反向转换，把元组转换成打包的字节序列。可以处理bytes、bytearray、memoryview对象

In [ ]:
import struct
fmt="<3S3SHH"
with open( 'filter.gif','rb') as fp:
	img = memoryview(fp.read())
header = img[ :10]
bytes(header) 
#b’GIF89a+\x02 xe6\x0a'
struct.unpack(fmt, header) 
#(b’GIF'， b'89a', 555， 230)


## 4.3编码器
1. 编码：文本转换成字节序列
2. 解码：字节序列转换成文本
## 4.4编解码问题
1. 个别编码方式存在无法编码特殊字符时，使用该编码进行解码或编码时会出现编码错误，通过指定string.encode/decode('',error='?')可以处理无法编码的字符,感知错误  

2. BOM
    + UTF-16 编码开头有额外的字节为BOM(U+FEFF 字节序标记→小字节序系统编码为b'\xff\xfe') 。指明编码时使用Intel CPU的小字节序还是大字节序  
    
    + 小字节序设备中，各个码位的最低有效字节在前面，如'E'码位U+0045，在字节偏移的第2位和第3位编码为69 和 0。+ 在大字节序 CPU中，编码顺序是相反的。如'E'编码为0 69
    +UTF-16 有两个变种：UTF-16LE，显式指明使用小字节序；UTF-16BE，显式指明使用大字节序。如果使用这两个变种，不会生成 BOM  
    
    + 根据标准，如果文件使用 UTF-16 编码，而且没有 BOM，那么应该假定它使用的是 UTF-16BE（大字节序）编码。然而，Intel x86 架构用的是小字节序，因此有很多文件用的是不带 BOM 的小字节序 UTF-16 编码。

In [7]:
E_u16 = 'E'.encode('utf_16')
print(E_u16)
print(list(E_u16))
E_u16le = 'E'.encode('utf_16le')
print(E_u16le)
print(list(E_u16le))
E_u16be = 'E'.encode('utf_16be')
print(E_u16be)
print(list(E_u16be))

b'\xff\xfeE\x00'
[255, 254, 69, 0]
b'E\x00'
[69, 0]
b'\x00E'
[0, 69]


## 4.5 处理文本文件
1. Unicode三明治——尽量不直接处理字节序列
    + 在输入的字节序列解码成字符串
    + 只处理文本字符串
    + 输出时编码成字节序列

## 4.6 为了正确比较而规范化Unicode字符
1. Unicode 有组合字符。如café有两种方式构成
2. 因组合字符的存在，可能个别字符看似相等（如é 与 e\u0301），实际上码位序列不同，因此在比较字符串时需要将其规范化。即unicodedata.normalize
    + NFC (Normalization Form C)使用最少的码位构成等价的字符串。
    + NFD 把组合字符分解成基字符和单独的组合字符


In [12]:
s1 = 'café'
s2 = 'cafe\u0301'
print(s1,s2)
print(len(s1),len(s2))
print(s1==s2)

from unicodedata import normalize
print(len(normalize('NFC',s1)), len(normalize('NFC',s2)))
print(normalize('NFC',s1) == normalize('NFC',s2))

print(len(normalize('NFD',s1)), len(normalize('NFD',s2)))
print(normalize('NFD',s1) == normalize('NFD',s2))



café café
4 5
False
4 4
True
5 5
True


In [20]:
from unicodedata import normalize,name
""" 使用NFC时。有些单字符会被规范成另一个单字符。因此在比较匹配要规范化，防止出现意外： """
ohm = '\u2126'
print(name(ohm), ohm)
ohm_c = normalize('NFC', ohm)
print(name(ohm_c),ohm_c)
print(ohm == ohm_c)
print(normalize('NFC', ohm) == normalize('NFC', ohm_c))

OHM SIGN Ω
GREEK CAPITAL LETTER OMEGA Ω
False
True


3. 大小写折叠 str.casefold() str.lower()
4. 去除变音符号 unicodedata.combining 返回符号的规范组合类作为整数，没有则返回0
5. 字符串映射，str.maketrans()

In [21]:
import unicodedata
def shave_marks(txt):
    """Remove all diacritic marks"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))  # <2>
    return unicodedata.normalize('NFC', shaved)  # <3>

order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

## 4.7 Unicode文本排序
1. local.strxfrm作为排序键
2. pyuca库
## 4.8 Unicode数据库
1. Unicode 标准提供了一个完整的数据库（许多格式化的文本文件），不仅包括码位与字符名称之间的映射，还有各个字符的元数据，以及字符之间的关系。
2. 方法
    + unicodedata.name()
    + unicodedata.numeric()
    + .isdecimal()
    + .isnumeric()

In [22]:
import unicodedata
import re

re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print('U+%04x' % ord(char),                       # <1>
          char.center(6),                             # <2>
          're_dig' if re_digit.match(char) else '-',  # <3>
          'isdig' if char.isdigit() else '-',         # <4>
          'isnum' if char.isnumeric() else '-',       # <5>
          format(unicodedata.numeric(char), '5.2f'),  # <6>
          unicodedata.name(char),                     # <7>
          sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


## 4.9 支持字符串和字节序列的双模式API
1. 正则表达式有字符串模式、字节序列模式
    + 如果使用字节序列构建正则表达式，\d 和\w 等模式只能匹配 ASCI 字符；
    + 如果是字符串模式，就能匹配 ASCI 之外的 Unicode 数字或字母。
    + 字节序列只能用字节序列正则表达式搜索   
    
2. os，给os模块函数传字节序列参数，可以得到字节序列返回值。为了便于手动处理字符串或字节序列形式的文件名或路径名，os模块提供了特殊的编码和解码函数。
    + fsencode(filename) 如果 filename 是 str 类型（此外还可能是 bytes 类型），使用sys.getfilesystemencoding(）返回的编解码器把 filename 编码成字节序列；否则，返回未经修改的 filename 字节序列。  

    + fsdecode(filename)如果 filename 是 bytes 类型（此外还可能是 str 类型），使用sys.getfilesystemencoding(）返回的编解码器把 filename 解码成字符串；否则，返回未经修改的 filename字符串。

In [23]:
import os
print(os.listdir('.'))
print(os.listdir(b'.'))

['4.文本与字节序列.ipynb', '5.一等函数.ipynb', '.DS_Store', '7.函数装饰器和闭包.ipynb', '1.Python数据模型.ipynb', 'example-code-master', '.history', 'README.md', '.ipynb_checkpoints', '9.符合Python风格的对象.ipynb', '.git', '8.对象引用、可变性和垃圾回收.ipynb']
[b'4.\xe6\x96\x87\xe6\x9c\xac\xe4\xb8\x8e\xe5\xad\x97\xe8\x8a\x82\xe5\xba\x8f\xe5\x88\x97.ipynb', b'5.\xe4\xb8\x80\xe7\xad\x89\xe5\x87\xbd\xe6\x95\xb0.ipynb', b'.DS_Store', b'7.\xe5\x87\xbd\xe6\x95\xb0\xe8\xa3\x85\xe9\xa5\xb0\xe5\x99\xa8\xe5\x92\x8c\xe9\x97\xad\xe5\x8c\x85.ipynb', b'1.Python\xe6\x95\xb0\xe6\x8d\xae\xe6\xa8\xa1\xe5\x9e\x8b.ipynb', b'example-code-master', b'.history', b'README.md', b'.ipynb_checkpoints', b'9.\xe7\xac\xa6\xe5\x90\x88Python\xe9\xa3\x8e\xe6\xa0\xbc\xe7\x9a\x84\xe5\xaf\xb9\xe8\xb1\xa1.ipynb', b'.git', b'8.\xe5\xaf\xb9\xe8\xb1\xa1\xe5\xbc\x95\xe7\x94\xa8\xe3\x80\x81\xe5\x8f\xaf\xe5\x8f\x98\xe6\x80\xa7\xe5\x92\x8c\xe5\x9e\x83\xe5\x9c\xbe\xe5\x9b\x9e\xe6\x94\xb6.ipynb']
